
# <p style="text-align:center"> <font color='darkorange'>**CUNEF**</font>
## <p style="text-align:center"> **TFM - Análisis de sentimiento pólitico en Twitter**
### <p style="text-align:center"> **3. Predicciones con GPT3**</strong><br />
    
<p style="text-align:left">Pablo Mazariegos Reviriego - <font color='orange'>pablo.mazariegos@cunef.edu </font>

En este proyecto de Trabajo Fin de Máster, realizaremos un análisis de sentimiento de los tweets hechos por los 5 candidatos políticos a la presidencia de Madrid durante el período de campaña política que abarcó desde el 12 hasta el 27 de mayo de 2023. Utilizaremos una base de datos recopilada manualmente que contiene los tweets de los candidatos. El objetivo principal de este proyecto es desarrollar modelos de aprendizaje automático que puedan clasificar los tweets según su sentimiento (positivo, negativo o neutral).

El proyecto se organizará en diferentes cuadernos, cada uno enfocado en una etapa específica del proceso:

 1. EDA y Selección/Preparación de variables
 2. Word Cloud y Análisis de viralidad
 3. <font color='darkgreen'> **Predicciones con GPT3**</font>

Este cuaderno se enfoca 

  **INDICE:**
 
 - [Importación de Librerias](#0) 
 - [Funciones utilizadas en este notebook](#1) 
 - [Carga de datos](#2)
 - [Exploración de los datos](#3)

In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import json
import pandas as pd
import openai
import string
import requests
import random

import warnings
warnings.filterwarnings('ignore')

In [ ]:
openai.organization = ""
openai.api_key = ""
#openai.Engine.list()

## Step 3 : CLI data preparation tool

In [8]:
df = pd.read_parquet('../data/processed/df_sentimiento_final.parquet')
df.head()

,partido,candidato,nick,followers,fecha,post,post_clean,sentimiento,sentimiento_num,negative,...,comments vs viewed,shared,shared vs viewed,likes,likes vs viewed,votos,porcentaje,escaños,Num_words_POST,sentimiento_clean
0,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Comenzamos la campaña, una vez más, junto a la...",comenzamos la campaa una vez ms junto a la agr...,Positive,1,0.034475,...,0.003373,261,0.004446,1260,0.021465,1586985,0.4734,71,42,comenzamos campaa vez ms junto agricultura cam...
1,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Madrid es la región del Espíritu de Ermua, la ...",madrid es la regin del espritu de ermua la que...,Negative,-1,0.904359,...,0.005413,561,0.005522,2127,0.020935,1586985,0.4734,70,24,madrid regin espritu ermua admite totalitarios...
2,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-12,"Majadahonda con ganas de Libertad, familia, un...",majadahonda con ganas de libertad familia unid...,Positive,1,0.014554,...,0.002345,213,0.003568,1042,0.017454,1586985,0.4734,70,18,majadahonda ganas libertad familia unidad aleg...
4,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-13,Presidente: líbranos del mal.,presidente lbranos del mal,Negative,-1,0.680318,...,0.002700,549,0.001660,2592,0.007836,1586985,0.4734,70,4,presidente lbranos mal
5,PP,Isabel Díaz Ayuso,@IdiazAyuso,912100,2023-05-13,En Madrid sólo habrá 2 tarifas de transporte p...,en madrid slo habr 2 tarifas de transporte pbl...,Negative,-1,0.628112,...,0.001445,759,0.003562,3016,0.014153,1586985,0.4734,70,51,madrid slo habr 2 tarifas transporte pblico ma...


## Step 4 : Create a fine-tuned model

In [ ]:
%env OPENAI_API_KEY=sk-wwXXXXXXXXXXXXXXXXXXXXXXXXXX

!openai api fine_tunes.create -t "data-tomato_prepared.jsonl" -m "curie"

In [ ]:
# List all created fine-tunes
!openai api fine_tunes.list

## Step 5 : Use a fine-tuned model

In [ ]:
YOUR_MODEL = "curie:ft-datamarketinglabs-2021-07-22-12-03-32"
YOUR_PROMPT = "Tomato description\n\nnom:Tomate Terrasi\n\ncalibre:très gros fruit - masse 300g à 1kg - calibre supérieur à 85mm\n\norigine:Russie\n\nhauteur:comprise entre 1.2 et 2 mètres\n\nusage:apte à la cuisson - coulis - sauce\n\nproduction:bonne\n\ncouleur:rouge\n\nmaladie:assez bonne résistance aux maladies - oïdium - mildiou\n\nprecocite:Mi saison de 65 à 80 jours\n\ndescription:"

In [ ]:
!openai api completions.create -m YOUR_MODEL -p YOUR_PROMPT

In [ ]:
def completion(prompt, engine_id, debug=True, **kwargs):

    COMPLETION_ENDPOINT = 'https://api.openai.com/v1/completions'
    
    headers = {'Authorization': 'Bearer {api_key}'.format(api_key=openai.api_key),
              'Content-Type': 'application/json'}

    data = {
              "prompt": prompt,
              "model": engine_id,
              "max_tokens": kwargs.get('max_tokens', 200),
              "temperature": kwargs.get('temperature', 0.3),
              "frequency_penalty": kwargs.get('frequency_penalty', 0.9),            
              "stop": [""]
            }
    
    data.update(kwargs)

    response = requests.post(COMPLETION_ENDPOINT, headers=headers, data=json.dumps(data))
    result = response.json()

    if debug:
        print('Headers:')
        print(json.dumps(headers, indent=4))
        print('Data:')
        print(json.dumps(data, indent=4))
        print('Result:')
        print(json.dumps(result, indent=4))

    if response.status_code == 200:
        return [x['text'].strip() for x in result['choices']]
    else:
        return "Error: {}".format(result['error']['message'])
  
completion(YOUR_PROMPT, YOUR_MODEL, debug=False)

In [ ]:
YOUR_PROMPT = "Tomato description\n\nnom:Tomate Babbardi\n\ncalibre:petit fruit - masse 30g à 100g - calibre inférieur à 45mm\n\norigine:France\n\nhauteur:comprise entre 1.2 et 1.5 mètres\n\nusage:purée\n\nproduction:bonne\n\ncouleur:rouge\n\nmaladie:assez bonne résistance aux maladies - covidou\n\nprecocite:Mi saison de 5 à 15 jours\n\ndescription:"

completion(YOUR_PROMPT, YOUR_MODEL, debug=False)